In [1]:
from collections import defaultdict

import numpy as np
import pandas as pd

In [2]:
#get answers from csv
a_dict = dict()
answers = pd.read_csv('answers.csv')
questions = answers.loc[:,"Example Scenario in Survey"]
correct = answers.loc[:,"Correct Answer"]
for i in range(0, len(questions)):
    cq = str(questions[i]).strip()
    ca = str(correct[i]).strip()
    a_dict[cq] = str(ca)

In [3]:
#dataframe raw responses
df = pd.read_csv('responses.csv')
#optional questions should not be collected
ignore = {"Q200", "Q629", "Q1051", "Q227", "Q1052", "Q254.1", "Q549", "Q1053", "Q401.1",  "Q202", "Q229"}
#drop QID row
df = df.drop(labels=1, axis=0)

#match question ids to question text
descriptionsDict = dict()
start = df.columns.get_loc("Q176")
end = df.columns.get_loc("QID1054")
for i in range(start, end + 1, 1):
    key = df.columns[i]
    if key not in ignore:
        descriptionsDict[key] = df.iat[0, i]

In [4]:
optional = set()
for question in ignore:
    for j in range(1, len(df.iloc[:, 0])):
        col = df.columns.get_loc(question)
        response = df.iat[j, col]
        if(str(response) != "nan") and ("no" not in str(response).lower()):
            optional.add(response)
print(optional)




{'nither', 'Too much scrolling past all the same options for every question. It would be better if the options are listed with radio buttons in a fixed area with a smaller lower area containing the questions. After selecting the answer and clicking forward button then the buttons are cleared and the next question appears. The user should be able to press a back arrow button in case they want to change a previous answer. Then a submit button at the end to send all the answers.', 'the questions were easy to understand', "Questions regarding login using email address I felt could be considered as App Functionality or Fraud Prevention or Account Management. So that wasn't entirely clear as it depends how you interpret the need.", 'When sending data to a third party server, I have assumed that my app would and could also collect this.', 'I did get a bit confused between some of the ones for app functionality and personalization. But overall I found it pretty clear and straightforward', 'The

In [5]:
#categorize questions by type
section1 = set()
section2 = set()
section3 = set()
attentionChecks = set()
for question in descriptionsDict:
    if 'attention check. Please go ahead and select the option that begins with "N".' in descriptionsDict[question]:
        section1.add(question)
        attentionChecks.add(question)
    if 'attention check. Please go ahead and select the option that begins with "H".' in descriptionsDict[question]:
        section2.add(question)
        attentionChecks.add(question)
    if 'attention check. Please go ahead and select the option that begins with "P".' in descriptionsDict[question]:
        section3.add(question)
        attentionChecks.add(question)
    if "collection and sharing" in descriptionsDict[question]:
        section1.add(question)
    if "user data type" in descriptionsDict[question]:
        section2.add(question)
    if "Why" in descriptionsDict[question]:
        section3.add(question)
typeDict = dict()
count = 0
start = df.columns.get_loc("Q176")
end = df.columns.get_loc("QID1054")
for i in range(start, end + 1, 1):
    key = df.columns[i]
    if key in ignore:
        continue
    if key in section1:
        entry = 1
    elif key in section2:
        entry = 2
    elif key in section3:
        entry = 3
    typeDict[key] = entry

In [6]:
#categorize questions by case
categoryDict = dict()
for question in typeDict:
    if typeDict[question] == 1:
        if "Our app accesses GPS location and sends it to our server." in descriptionsDict[question]:
             categoryDict[question] = "collect basic"
        elif "Our app accesses GPS location and sends it to a third party server." in descriptionsDict[question]:
            categoryDict[question] = "share basic"
        elif "users to navigate the open web via a webview. When the user opens google.com, it accesses the user's GPS location and sends the GPS location to Google's server" in descriptionsDict[question] or "Our app accesses and processes the user's GPS location directly on the user’s device. The data never leaves the device" in descriptionsDict[question] or "Our app encrypts the user’s message before it is sent to another user, where it is decrypted" in descriptionsDict[question]:
            categoryDict[question] = "collect exempt"
        elif "Our app allows users to navigate the open web via a webview. When the user opens google.com, it accesses the user's GPS location and sends the GPS location to a third party server" in descriptionsDict[question] or "Our app accesses GPS location and sends it off the device to Google Maps to get real-time traffic updates" in descriptionsDict[question] or "Our app accesses GPS location and stores it on Google's server using Google Cloud Storage" in descriptionsDict[question] or "Our app accesses the user’s GPS location and sends it to government agencies in response to a legal request" in descriptionsDict[question] or "Our app's user explicitly initiates an action that sends their full name to a third party server" in descriptionsDict[question] or "Our app sends the user's full name to a third party server after obtaining the user's consent to it" in descriptionsDict[question] or "Our app sends the user's medical records that have been fully anonymized to a third party server." in descriptionsDict[question]:
            categoryDict[question] = "share exempt"
        elif "Our app accesses GPS location and sends it to our server via libraries and/or SDKs included in our app" in descriptionsDict[question] or "website then accesses the user's GPS location through the webview, and sends it to our server" in descriptionsDict[question] or "Our app sends the user's GPS location to our server to search for nearby places but does not store the GPS location after serving the request" in descriptionsDict[question] or "Our app requests for the user's full name and only sends the pseudonymized name to our server" in descriptionsDict[question] or "Our app encrypts the user’s bank account information and sends it to our server" in descriptionsDict[question] or "Our app asks for a user's birthday for marketing communication, but that info is not required" in descriptionsDict[question]:
            categoryDict[question] = "collect special"
        else:
            categoryDict[question] = "share special"
    else:
        categoryDict[question] = "basic"

In [7]:
#get correct answers for each question (based on ID)
correctAnswerDict = dict()
#question names
nameDict = dict()
for question in descriptionsDict:
    search_key = descriptionsDict[question]
    if "Data usage scenario:" in search_key:
        search_key = search_key.replace("Data usage scenario:", "")
    if "Our app collects:" in search_key:
        search_key = search_key.replace("Our app collects:", "")
    if "Select" in search_key:
        end = search_key.index("Select")
        search_key = search_key[1:end-1].strip()
    elif "Why" in search_key:
        end = search_key.index("Why")
        search_key = search_key[1:end-1].strip()
    elif "Is" in search_key:
        end = search_key.index("Is")
        search_key = search_key[1:end-1].strip()
    else:
        end = len(search_key)
        search_key = search_key[1:end-1].strip()
    if search_key[-1] == ".":
        search_key = search_key[:-1]
    if search_key in a_dict:
        correctAnswerDict[question] = a_dict[search_key]
        nameDict[question] = search_key.replace("\n", "")
    else:
        for key in a_dict:
            if search_key in key:
                    correctAnswerDict[question] = a_dict[key]
                    break
        nameDict[question] = key.replace("\n", "")
    if "Please" in nameDict[question]:
        nameDict[question] = nameDict[question].replace("Please" , " Please").strip()

In [8]:
#potential answers, for indexing
collection = ['Collected', 'Shared', 'Both', 'Neither']
types = ['Location', 'Personal Information', 'Financial Information', 'Health and Fitness', 'Messages', 'Photos or Videos', 'Audio Files', 'Files and docs', 'Calendar', 'Contacts', 'App Activity', 'Web Browsing', 'App Information and Performance', 'Device or Other Identifiers', 'None of the Above']
purpose = ['App functionality', 'Analytics', 'Developer communications', 'Advertising or marketing', 'Fraud prevention, security, and compliance', 'Personalization', 'Account management', 'None of the Above']
ignore = {"Q200", "Q629", "Q1051", "Q227", "Q1052", "Q254.1", "Q549", "Q1053", "Q401.1",  "Q202", "Q229"}

#calculating index of correct answer
answerDict = defaultdict(str)
indexGivenDict = defaultdict(str)
indexExpectedDict = defaultdict(str)
rowIds = []
start = df.columns.get_loc("Q176")
end = df.columns.get_loc("QID1054")
question1Count = dict()
question2Count = dict()
question3Count = dict()
#participants who did not pass the attention check
remove = set()
count = 0
start = df.columns.get_loc("Q176")
end = df.columns.get_loc("QID1054")
for i in range(start, end+1, 1):
    for j in range(1, len(df.iloc[:, 0])):
        participantId = df.iat[j, df.columns.get_loc("Q162")]
        questionId = df.columns[i]
        if questionId in ignore:
            continue
        rowId = (participantId, questionId)
        if questionId not in attentionChecks:
            rowIds.append(rowId)
        answer = str(df.iat[j, i])
        if answer == "nan":
            answerDict[rowId] = ""
            continue
        answer = str(answer).strip()
        if answer in collection or answer in types or answer in purpose:
            answer = answer
        else:
            if typeDict[questionId] == 1:
                for word in collection:
                    if answer[0:6] in word:
                        answer = word
            elif typeDict[questionId] == 2:
                for word in types:
                    if answer[0:9] in word:
                        answer = word
            elif typeDict[questionId] == 3:
                for word in purpose:
                    if answer[0:9] in word:
                        answer = word
        if answer not in collection and answer not in types and answer not in purpose:
            continue
        answerDict[rowId] = str(answer)
        #remove questions that don't pass the attentionChecks
        if questionId in attentionChecks:
            if answer != correctAnswerDict[questionId]:
                remove.add(participantId)
                print(participantId)
                print(answer)
                print(questionId)
            if typeDict[questionId] == 1:
                question1Count[participantId] = question1Count.get(participantId, 0) + 1
            elif typeDict[questionId] == 2:
                question2Count[participantId] = question2Count.get(participantId, 0) + 1
            else:
                question3Count[participantId] = question3Count.get(participantId, 0) + 1
            continue
        #look at difference between expected index and given index
        if typeDict[questionId] == 1:
            index_given = collection.index(answer)
            index_expected = collection.index(correctAnswerDict[questionId])
            question1Count[participantId] = question1Count.get(participantId, 0) + 1
        elif typeDict[questionId] == 2:
            index_given = types.index(answer)
            index_expected = types.index(correctAnswerDict[questionId])
            question2Count[participantId] = question2Count.get(participantId, 0) + 1
        else:
            index_given = purpose.index(answer)
            index_expected = purpose.index(correctAnswerDict[questionId])
            question3Count[participantId] = question3Count.get(participantId, 0) + 1
        indexGivenDict[rowId] = index_given
        indexExpectedDict[rowId] = index_expected
#make sure user answered required amount of questions
for participant in question1Count:
    if question1Count[participant] < 11:
        print(participant)
        print(question1Count[participant])
        remove.add(participant)
    if participant not in question2Count or question2Count[participant] != 16:
        print(participant)
        print(0 if participant not in question2Count else question2Count[participant])
        remove.add(participant)
        continue
    if participant not in question3Count or question3Count[participant] != 16:
        print(participant)
        print(0 if participant not in question3Count else question3Count[participant])
        remove.add(participant)

59ea4e1116ecc200016ff81e
Shared
Q176
57309f6b30a556000fecdd47
None of the Above
Q554
6113a86e9e254bad97528743
App functionality
Q944
606540bd35c28c87db3e2ef9
0


In [9]:
#make sure user answered all eligibility questions
for j in range(1, len(df.iloc[:, 0])):
    participantId = df.iat[j, df.columns.get_loc("Q162")]
    ageQuestion = df.columns.get_loc("Q4")
    expQuestion = df.columns.get_loc("Q102")
    if str(df.iat[j, ageQuestion]) != "Yes" or str(df.iat[j, expQuestion]) != "Yes":
        print(str(df.iat[j, ageQuestion]))
        print(str(df.iat[j, expQuestion]))
        remove.add(participantId)
        print(participantId)

Yes
5
56435b83942576000a66eb5d
Yes
5
612520bd6605794b27e6a0fd
Yes
5
5ad0f1934d061300011b0a4d
Yes
5
602454a47d0d7d121a1e444b
Yes
5
5e6c0b0fede0271d08d1a3cd
Yes
5
5fa23c9b64b18a4067cc842e
Yes
5
6025ba2ad22ee81ad428c38c
Yes
5
610ce0a48fc7e68abc4070c1
Yes
nan
61015f63f9a351d3171a0f98
Yes
nan
607dc10ae5bc5c8b32b18e9a


In [10]:
#get condition
conditionDict = dict()
for j in range(0, len(df.iloc[:, 0]), 1):
    conditionDict[df.iat[j, df.columns.get_loc("Q162")]] = df.iat[j, -1]
conditionDict.pop('What is your Prolific ID?')

balanced = dict()
for person in conditionDict:
    balanced[conditionDict[person]] = balanced.get(conditionDict[person], 0) + 1
print(balanced)

for participant in remove:
    print(conditionDict[participant])
print(remove)

{'2': 47, '3': 47, '1': 49}
2
3
2
3
2
1
2
3
3
3
2
1
1
3
{'5fa23c9b64b18a4067cc842e', '6113a86e9e254bad97528743', '606540bd35c28c87db3e2ef9', '6025ba2ad22ee81ad428c38c', '607dc10ae5bc5c8b32b18e9a', '59ea4e1116ecc200016ff81e', '612520bd6605794b27e6a0fd', '5e6c0b0fede0271d08d1a3cd', '61015f63f9a351d3171a0f98', '5ad0f1934d061300011b0a4d', '56435b83942576000a66eb5d', '602454a47d0d7d121a1e444b', '57309f6b30a556000fecdd47', '610ce0a48fc7e68abc4070c1'}


In [11]:
#generating dataframe columns
results = pd.DataFrame()
participantIds = [p for p, q in rowIds]
questionIds = [q for p, q in rowIds]
descriptions = [descriptionsDict[q] for q in questionIds]
types = [typeDict[q] for q in questionIds]
categories = [categoryDict[q] for q in questionIds]
answers = [answerDict[(p, q)] for p, q in zip(participantIds, questionIds)]
correctAnswers = [correctAnswerDict[q] for q in questionIds]
conditions = [conditionDict[p] for p in participantIds]
givenIndices = [indexGivenDict[(p, q)] for p, q in zip(participantIds, questionIds)]
expectedIndices = [indexExpectedDict[(p, q)] for p, q in zip(participantIds, questionIds)]
names = [nameDict[q] for q in questionIds]
results['Question ID'] = questionIds
results['Participant ID'] = participantIds
results['Description'] = descriptions
results['Answer'] = answers
results['Correct Answer'] = correctAnswers
results['Question Type'] = types
results["Question Category"] = categories
results['Question Name'] = names
results['Prompt Condition'] = conditions
results['Answer Index'] = givenIndices
results['Correct Answer Index'] = expectedIndices
# filter out rows with empty answers (i.e., not selected in the study)
results = results[results["Answer"] != ""]
#remove rows that don't pass the attentionChecks
results = results[~results["Participant ID"].isin(remove)]
results['Correctness'] = (results['Answer'] == results['Correct Answer']).apply(lambda v: 1 if v else 0)

results['Order Difference'] =\
    (results['Answer Index'] - results['Correct Answer Index']).apply(lambda v: 0 if v == 0 else 1 if v > 0 else -1)

In [12]:
#export to csv
results.to_csv("output.csv")

In [13]:
print(results["Prompt Condition"].value_counts())

1    1866
3    1622
2    1621
Name: Prompt Condition, dtype: int64
